In [23]:
# load transformers and pandas
file_path = './artwar.1b.txt'

with open(file_path, 'r') as f:
    book_text = f.read()
    book_text = book_text.lower()
    
dataset= book_text.split('\n')

# save this

with open('./artwar.1b_cleaned.txt', 'w') as f:
    f.write(book_text)

In [24]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load the pretrained model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Tokenize the text (this assumes that 'the_art_of_war.txt' is your text file)
file_path = dataset
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="./artwar.1b_cleaned.txt",
    block_size=128
)

# Data collator used for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False  # For GPT2, we use mlm=False. It's not a masked language model.
)

# Initialize Trainer
training_args = TrainingArguments(
    output_dir='./results',          # output directory for model checkpoints
    overwrite_output_dir=True,       # overwrite the content of the output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=4,   # batch size for training
    save_steps=10_000,               # after # steps model is saved
    save_total_limit=2,              # only last # models are saved
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained('./finetuned_model')
tokenizer.save_pretrained('./finetuned_model')

e:\Users\hcang\anaconda3\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


  0%|          | 0/90 [00:00<?, ?it/s]

{'train_runtime': 14.5152, 'train_samples_per_second': 24.802, 'train_steps_per_second': 6.2, 'train_loss': 3.1182271321614583, 'epoch': 3.0}


('./finetuned_model\\tokenizer_config.json',
 './finetuned_model\\special_tokens_map.json',
 './finetuned_model\\vocab.json',
 './finetuned_model\\merges.txt',
 './finetuned_model\\added_tokens.json')

In [28]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load fine-tuned model and tokenizer
model_path = './finetuned_model'
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# Encode a text prompt to start off the generation
input_text = "The supreme art of war is"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate text using the model
max_length = 50  # specifies how long the generated passage should be
sample_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=max_length,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3
)

# Decode the generated text
# for i, sample_output in enumerate(sample_outputs):
#     print(f"Sample {i+1}: {tokenizer.decode(sample_output, skip_special_tokens=True)}\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sample 1: The supreme art of war is to use both sides in a single battle and to occupy them by an overwhelming and constant rout.

Sample 2: The supreme art of war is the defense and defense of his country
or country from attack.

Sample 3: The supreme art of war is to fight for victory; but if the enemy has not a body or soldiers, his army is like a jumble; and if the enemy has a great number of men, his army is like a flock, and the.

